In [ ]:
import torch
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BitsAndBytesConfig

from utils import pic2float

In [2]:
import sys, os

# добавить папку уровнем выше (корень проекта) в PYTHONPATH
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import torch
from transformers import GroundingDinoProcessor, GroundingDinoForObjectDetection, BitsAndBytesConfig
from torch.amp import autocast
from PIL import Image

from utils import pic2float, device, memo

In [3]:
class GDINO:
    def __init__(self, model_id="IDEA-Research/grounding-dino-tiny", device="cuda"):
        self.device = device

        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,                # включаем 4-битную квантизацию
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4"         # можно попробовать и другие варианты, например 'fp4'
        )

        self.processor = GroundingDinoProcessor.from_pretrained(model_id)
        self.model = GroundingDinoForObjectDetection.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            quantization_config=quant_config,
            low_cpu_mem_usage=True,
        ).to(device)

    def detect_objects(self, image, text='a cat'):
        image = pic2float(image)
        text = preprocess_caption(text)
        inputs = self.processor(
            image,
            return_tensors="pt",
            text=text,
            do_rescale=False
        ).to(self.device)

        with torch.no_grad():
            with autocast('cuda', dtype=torch.float16):
                outputs = self.model(**inputs)

        results = self.processor.image_processor.post_process_object_detection(
            outputs,
            target_sizes=[image.shape[:-1]],
            threshold=0.1
        )[0]
        return results

print('Loading GDINO model...')
gdino = GDINO()
print('GDINO model loaded')

Loading GDINO model...


KeyboardInterrupt: 

In [11]:
model_id="IDEA-Research/grounding-dino-tiny"
quant_config = BitsAndBytesConfig(
            load_in_4bit=True,                # включаем 4-битную квантизацию
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4"         # можно попробовать и другие варианты, например 'fp4'
        )

processor = GroundingDinoProcessor.from_pretrained(model_id)

model = GroundingDinoForObjectDetection.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

n:\PROJECTS\python\STUDY\tg_pet_detector\tg_pet_detector_GUI\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\_TEMP\model\HF_HOME\hub\models--IDEA-Research--grounding-dino-tiny. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/689M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [17]:
import sys, torch, transformers
print("python:", sys.executable)
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)

python: n:\PROJECTS\python\STUDY\tg_pet_detector\tg_pet_detector_GUI\.venv\Scripts\python.exe
torch: 2.5.1+cu124
transformers: 4.48.1


In [1]:
from transformers import GroundingDinoProcessor, GroundingDinoForObjectDetection
import torch

model_id = "IDEA-Research/grounding-dino-tiny"
device = "cuda" if torch.cuda.is_available() else "cpu"

print("loading processor...")
processor = GroundingDinoProcessor.from_pretrained(model_id, local_files_only=True)
print("processor ok")

print("loading model...")
model = GroundingDinoForObjectDetection.from_pretrained(model_id, local_files_only=True)
print("model loaded on CPU")

model = model.to(device)
print("moved to", device)

loading processor...
processor ok
loading model...
model loaded on CPU
moved to cuda
